In [1]:
import pandas as pd

/tmp/ipykernel_1199996/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
import arrow
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
#from langchain.chains.summarize import load_summarize_chain
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
import re
import pandas as pd
import sys
sys.path.append("..")
from utils.data_utils import read_data
from utils.log_setting import logger
from prompt.base import prompt_dict

load_dotenv()
output_parser = StrOutputParser()
summarization_prompt_tmpl = (
    "請遵守以下的【規範】，根據【篩選資料】、【GRI 準則】及【偏好語氣】，總結為一段可以當作 ESG 報告草稿的摘要。\n\n"
    "【規範】\n"
    "- 摘要必須不少於 500 字。\n"
    "- 請依照 GRI 準則的項目順序產生摘要。\n"
    "\n"
    "####\n"
    "開始!\n"
    "【篩選資料】\n"
    "{filtered_data}\n"
    "【GRI 準則】\n"
    "{gri_standard}"
    "【偏好語氣】\n"
    "{tone}\n"
    "包含公司名稱\"
    "包含報告年份\"
    "包含物料類: 301-1 所用物料的重量或體積、301-2 使用回收再利用的物料\"
    "包含物料類: 301-3 回收的產品及其包材\"
    "包含能源類: 302-1 組織內部的能源消耗量、302-2 組織外部的能源消耗量\"
    "包含能源類: 302-3 能源密集度、302-4 減少能源消耗、302-5 降低產品和服務的能源需求\"
    "包含水和廢水類: 303-1 共享水資源之相互影響、303-2 與排水相關衝擊的管理\"
    "包含水和廢水類: 303-3 取水量、303-4 排水量、303-5 耗水量\"
    "包含生物多樣性類: 304-1 組織所擁有、租賃、管理的營運據點或其鄰近地區位於環境保護區或其他高生物多樣性價值的地區\"
    "包含生物多樣性類: 304-2 活動、產品及服務，對生物多樣性方面的顯著衝擊\"
    "包含生物多樣性類: 304-3 受保護或復育的棲息地、304-4 受營運影響的棲息地中，已被列入IUCN紅色名錄及國家保護名錄的物種\"
    "包含排放類: 305-1 直接（範疇一）溫室氣體排放、305-2 能源間接（範疇二）溫室氣體排放、305-3 其他間接（範疇三）溫室氣體排放\"
    "包含排放類: 305-4 溫室氣體排放密集度、305-5 溫室氣體排放減量、305-6 破壞臭氧層物質的排放、\"
    "包含排放類: 305-7 氮氧化物（NOx）、硫氧化物（SOx），及其他重大的氣體排放\"
    "包含廢棄物類: 306-1 廢棄物產出情形及廢棄物相關顯著衝擊、306-2 廢棄物相關顯著衝擊管理、306-3 廢棄物產出種類及量\"
    "包含廢棄物類: 306-4 廢棄物回收方式、種類與量（分廠內、廠外）、306-5 廢棄物處置方式、種類與量（分廠內、廠外）\"
    "包含合規類: 307-1 違反\"
    "包含環保法規: 308-1 採用環境標準篩選新供應商、308-2 供應鏈對環境的負面影響\"
    "請一步一步的摘要出公司的ESG報告，以確保報告完整正確。\"
    "偏好語氣: {friendly + professional, authoritative + inofrmative, urgent + persuasive, straigthforward + conversational, professional + thrusworthy, authoritative + expert, creative + professional}\"
    "input: {”reference_docs”: [], “query”: “<產出摘要的 prompt 要加入語調的變數>“,“tone“:“偏好語氣“\}"
)


def get_answer_normal(query, filtered_data, summarization_prompt_tmpl):
    if filtered_data is None:
        filtered_data = get_date_range_from_query(query)

    prompt = ChatPromptTemplate.from_template(summarization_prompt_tmpl)
    model = ChatOpenAI(model=os.getenv("OPENAI_MODEL_NAME"), temperature=0.75)
    chain = prompt | model | output_parser
    # ans = chain.invoke({"query": query, "filtered_data": filtered_data})

    generator = chain.stream({"query": query, "filtered_data": filtered_data})
#generator = chain.stream({"query": query, "filtered_data": filtered_data, "tone": tone}})
    return generator


if __name__ == "__main__":
    ans = get_answer_normal("台泥碳排量")
    print(ans)
